In [1]:
# Import necessary libraries;
import numpy as np
from numpy import mean
from numpy import std


import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set_style('white')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from mpl_toolkits.mplot3d import Axes3D
from numpy import genfromtxt
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.datasets import load_digits, make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn import tree
import plotly.express as px

In [2]:
fireWall = pd.read_csv('log2.csv')

fireWallXData = fireWall[['Source Port', 'Destination Port','NAT Source Port',
                         'NAT Destination Port','Bytes','Bytes Sent','Bytes Received','Packets',
                        'Elapsed Time (sec)','pkts_sent','pkts_received']]
fireWallyData = fireWall[['Action']]

X = fireWallXData.iloc[:,0:10]
y = fireWallyData.iloc[:,0]


print(y.shape)
print(X.shape)

#print(fireWall.shape)

FireWALLData = pd.concat([X, y], axis=1)
print(FireWALLData.shape)

newX = FireWALLData.iloc[:,0:9]
newy = FireWALLData.iloc[:,10]
FireWALLData.head()




(65532,)
(65532, 10)
(65532, 11)


,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,Action
0,57222,53,54587,53,177,94,83,2,30,1,allow
1,56258,3389,56258,3389,4768,1600,3168,19,17,10,allow
2,6881,50321,43265,50321,238,118,120,2,1199,1,allow
3,50553,3389,50553,3389,3327,1438,1889,15,17,8,allow
4,50002,443,45848,443,25358,6778,18580,31,16,13,allow


In [3]:

from sklearn.preprocessing import LabelEncoder
encodeLabel = LabelEncoder()
newy = encodeLabel.fit_transform(newy)
print(newy)
X_train, X_test, y_train, y_test = train_test_split( newX, newy, test_size=60000, random_state=1738)


[0 0 0 ... 2 2 2]


# RANDOM FORREST TRIAL ONE ON Fire Wall DATASET

In [4]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 25000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_1 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL ONE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   31.9s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   37.2s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   41.2s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   47.7s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   52.8s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   58.2s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL ONE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.7min finished


In [5]:
TRIAL_1.cv_results_

{'mean_fit_time': array([1.3811873 , 2.638169  , 5.28584528, 1.45465097, 2.79440346,
        5.48751922, 1.43863673, 2.83964248, 5.68078537, 1.43563433,
        3.03220782, 6.07212186, 1.99171228, 3.5970933 , 6.96619034,
        1.8253695 , 3.41853976, 6.89522986, 1.75911288, 3.52673244,
        7.68290744, 1.96008606, 4.04708033, 8.810077  , 1.5281136 ,
        3.66625252, 6.35026135, 1.70523539, 3.11848197, 6.18241658,
        1.5755549 , 3.40592847, 6.78943887, 1.70896964, 3.4379571 ,
        6.49298344, 2.16256037, 4.27527642, 7.284164  , 2.00012035,
        3.555057  , 6.8763073 , 1.76211534, 3.5340395 , 7.50565448,
        1.80695376, 3.84060283, 8.22687535, 1.68324766, 2.89468961,
        5.79168167, 1.56414495, 2.94963632, 5.85983987, 1.54783115,
        3.12478728, 6.06921949, 1.56294394, 2.94323139, 5.80639343,
        1.71957841, 3.44246063, 7.04495831, 1.9323617 , 3.66294985,
        7.53107667, 2.05326562, 4.1780931 , 7.02954593, 1.89132667,
        3.65394244, 5.52625217]

In [6]:
print("---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_neg_log_loss']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 9,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 100}

In [7]:
print("---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_f1_micro']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 11,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 400}

In [8]:
print("---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_roc_auc_ovo']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 11,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [9]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best NEG LOG LOSS hyperparameters :0.9978666666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best F1 hyperparameters :0.9977666666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best ROC_AUC hyperparameters :0.99785
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL TWO ON Fire Wall DATASET

In [12]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 25000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_2 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL TWO RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   34.0s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   38.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   54.4s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL TWO RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.6min finished


In [13]:
TRIAL_2.cv_results_

{'mean_fit_time': array([1.34525762, 2.67670255, 5.36421123, 1.42262249, 2.81131778,
        5.70400558, 1.47046456, 2.85755744, 5.74253864, 1.47096486,
        2.85035086, 5.82911315, 1.89362769, 3.64193187, 7.11121597,
        1.90443749, 3.67005606, 7.3992631 , 1.95378022, 3.81097765,
        7.74055662, 1.93286185, 3.83319664, 7.62355657, 1.4188199 ,
        2.87497263, 5.69169502, 1.51180029, 2.91220436, 6.03168745,
        1.60287876, 3.02490101, 6.12396698, 1.65512338, 3.19224553,
        6.71387329, 1.92315359, 3.80617304, 7.26104441, 2.06237335,
        3.73661313, 7.25463867, 1.78383398, 3.73581309, 7.50245204,
        2.0188364 , 3.65764523, 7.34802041, 1.3181335 , 2.69751978,
        5.39223709, 1.41481652, 2.85765743, 5.76836133, 1.46485953,
        2.87377167, 6.00256224, 1.55083365, 3.20956049, 5.92469487,
        1.82366815, 3.5502532 , 6.83617868, 1.82046595, 3.65003843,
        7.18207626, 1.82426848, 3.58828516, 6.78563533, 1.83998194,
        3.41663823, 5.39103646]

In [14]:
print("---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_neg_log_loss']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 9,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 400}

In [15]:
print("---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_f1_micro']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 7,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [16]:
print("---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_roc_auc_ovo']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 7,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 100}

In [17]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best NEG LOG LOSS hyperparameters :0.9979666666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best F1 hyperparameters :0.9980333333333333
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best ROC_AUC hyperparameters :0.9982333333333333
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL THREE ON Fire Wall  DATASET

In [18]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 25000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_3 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL THREE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   28.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   36.5s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   42.2s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   46.3s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   51.9s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL THREE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.5min finished


In [19]:
TRIAL_3.cv_results_

{'mean_fit_time': array([1.33494854, 2.67510133, 5.38503041, 1.43132987, 2.83313665,
        5.63834915, 1.46616087, 2.91000204, 5.83722   , 1.51039901,
        2.95634265, 5.73313012, 1.74650202, 3.38170829, 6.70866947,
        1.8056529 , 3.5213284 , 7.09249892, 1.81956482, 3.67005601,
        7.25063562, 1.87140942, 3.67365928, 7.40857148, 1.3255404 ,
        2.64647551, 5.37081881, 1.4157177 , 2.84214401, 5.67868381,
        1.49058251, 2.92481546, 5.79698486, 1.4858777 , 2.89599104,
        5.79518361, 1.84498634, 3.46117673, 6.72178078, 1.80034804,
        3.50611506, 7.07458415, 1.85409436, 3.78435445, 7.44770465,
        1.98630805, 3.71689625, 7.38835382, 1.32163706, 2.66809454,
        5.2845448 , 1.39970388, 2.83713937, 5.64075089, 1.47326703,
        2.89308815, 5.83111463, 1.52701302, 2.96214843, 5.87575269,
        1.79204073, 3.46568022, 6.80335059, 1.7765276 , 3.5181262 ,
        7.20409565, 1.89082551, 3.75042524, 7.13153377, 1.87841558,
        3.54564981, 5.60912333]

In [20]:
print("---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_neg_log_loss']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 9,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 100}

In [21]:
print("---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_f1_micro']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 11,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [22]:
print("---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_roc_auc_ovo']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 11,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 200}

In [23]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))


------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best NEG LOG LOSS hyperparameters :0.9979333333333333
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best F1 hyperparameters :0.99785
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best ROC_AUC hyperparameters :0.9977


# RANDOM FOREST TRIAL FOUR ON Fire Wall DATASET

In [24]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 25000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_4 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FOUR RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   32.0s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   40.6s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   44.7s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   50.3s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FOUR RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.5min finished


In [25]:
TRIAL_4.cv_results_

{'mean_fit_time': array([1.29621511, 2.62946138, 5.14932771, 1.38929501, 2.74215803,
        5.43737578, 1.44824586, 2.77618756, 5.63554649, 1.4363359 ,
        2.86546412, 5.58069892, 1.63110242, 3.28612599, 6.41711907,
        1.73799415, 3.40312662, 6.74640188, 1.7828331 , 3.51111975,
        6.99141269, 1.798947  , 3.57537498, 7.20539651, 1.2780993 ,
        2.55739903, 5.11309748, 1.35826788, 2.714434  , 5.46620064,
        1.47536926, 2.79800615, 5.56128268, 1.42802835, 2.87827554,
        5.64095044, 1.67914381, 3.31745319, 6.53091645, 1.72718515,
        3.40542822, 6.81426005, 1.85269265, 3.53634109, 7.03895335,
        1.77792902, 3.56526647, 7.2202095 , 1.26568871, 2.54839139,
        5.17955461, 1.38479071, 2.73365068, 5.46189685, 1.43353248,
        2.82382898, 5.5893075 , 1.46385856, 2.79690523, 5.72172027,
        1.70206413, 3.34897995, 6.78573508, 1.80635324, 3.67015657,
        7.0166337 , 1.87661357, 3.56816907, 6.80405145, 1.8225677 ,
        3.42724729, 5.45318928]

In [26]:
print("---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_neg_log_loss']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 9,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [27]:
print("---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_f1_micro']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 9,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [28]:
print("---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_roc_auc_ovo']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 11,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 100}

In [29]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best NEG LOG LOSS hyperparameters :0.9979666666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best F1 hyperparameters :0.9979166666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best ROC_AUC hyperparameters :0.9975666666666667
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL FIVE ON Fire Wall DATASET

In [30]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 25000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_5 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FIVE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   37.5s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   47.3s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   53.0s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FIVE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.6min finished


In [31]:
TRIAL_5.cv_results_

{'mean_fit_time': array([1.53011532, 2.8080143 , 5.62583766, 1.53011637, 2.95203805,
        5.73102875, 1.49238372, 2.95614295, 5.89286823, 1.50109105,
        2.96815243, 5.64485407, 1.69685907, 3.41944041, 7.01693482,
        1.8122582 , 3.56756778, 7.09850454, 1.91434593, 3.57027016,
        7.14254274, 1.8059536 , 3.56216335, 7.0178349 , 1.33735027,
        2.65868669, 5.78377461, 1.4423409 , 3.16121917, 6.1877213 ,
        1.63630753, 3.00558472, 5.76645942, 1.42602615, 2.85445433,
        5.92929869, 1.89042578, 3.74632158, 7.18137546, 1.84949021,
        3.59519138, 7.25563984, 1.85889845, 3.741117  , 7.06167302,
        1.78473496, 3.53864303, 7.44710517, 1.33925214, 2.79840727,
        5.95572162, 1.74710312, 3.18173614, 6.18882265, 1.82076583,
        3.28762779, 6.08102951, 1.60407929, 2.96004491, 6.02147822,
        1.83107514, 3.62271538, 8.04882116, 2.09620304, 4.65170016,
        9.27527676, 2.42208304, 4.53269815, 8.31645222, 2.57931805,
        4.04647961, 6.17541137]

In [32]:
print("---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_neg_log_loss']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 9,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 400}

In [33]:
print("---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_f1_micro']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 9,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 400}

In [34]:
print("---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_roc_auc_ovo']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 11,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 100}

In [35]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best NEG LOG LOSS hyperparameters :0.99785
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best F1 hyperparameters :0.9978666666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best ROC_AUC hyperparameters :0.9978833333333333
------------------------------------------------------------------------------------------------
